In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# this for the sake of helping the student

# benchmark_userclicks_7_18_10000_1000000_batch146_15_.csv

# remove feature 0 which is the time stamp
t = pd.read_csv('benchmark_userclicks_7_18_10000_1000000_batch146_15_.csv').iloc[:, 1:]
# cpu contention
# anomaly period: 1778 - 2378

print (t.shape)

(4218, 2283)


In [9]:
# clean data format
df = t.fillna(value = 0)

# normalization
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)
#train.to_csv('demo_train', index=False)
print (df.shape)

(4218, 2283)


In [11]:
train = pd.concat([df[0:1779], df[2979:]])
test = df[1779:2979]

print (train.shape)
print (test.shape)

(3018, 2283)
(1200, 2283)


In [12]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# construct a very simple auto-encoder
input_data = Input(shape=(train.shape[1],))
encoded = Dense(100, activation='relu')(input_data)
decoded = Dense(train.shape[1], activation='linear')(encoded)

autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='rmsprop', loss='mse')

# train
autoencoder.fit(train, train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(train[:100], train[:100]))

# save model
from keras.models import load_model
autoencoder.save('ae.h5')  # creates a HDF5 file 'my_model.h5'

/Users/feisong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 3018 samples, validate on 100 samples
Epoch 1/10
3018/3018 [==============================] - 0s 164us/step - loss: 0.1882 - val_loss: 0.0408
Epoch 2/10
3018/3018 [==============================] - 0s 118us/step - loss: 0.0271 - val_loss: 0.0264
Epoch 3/10
3018/3018 [==============================] - 0s 118us/step - loss: 0.0228 - val_loss: 0.0282
Epoch 4/10
3018/3018 [==============================] - 0s 118us/step - loss: 0.0237 - val_loss: 0.0276
Epoch 5/10
3018/3018 [==============================] - 0s 112us/step - loss: 0.0231 - val_loss: 0.0272
Epoch 6/10
3018/3018 [==============================] - 0s 102us/step - loss: 0.0241 - val_loss: 0.0283
Epoch 7/10
3018/3018 [==============================] - 0s 101us/step - loss: 0.0229 - val_loss: 0.0310
Epoch 8/10
3018/3018 [==============================] - 0s 88us/step - loss: 0.0224 - val_loss: 0.0278
Epoch 9/10
3018/3018 [==============================] - 0s 99us/step - loss: 0.0238 - val_loss: 0.0320
Epoch 10/10
3018/30

In [15]:
# do prediction on test set

test_result = autoencoder.predict(test)

# calculate the test error
def recons_error(input_set, recons):
    dif = input_set - recons
    dif = np.power(dif, 2)
    return np.mean(dif, axis=1)

reconstruction_error = recons_error(test_result, test)

# scale to 0-1, probability
ae = pd.DataFrame(np.array(reconstruction_error))
scaler_ae = MinMaxScaler()
ae = scaler_ae.fit_transform(ae)

for i in range(10, 20):
    print (ae[i])

[0.19060907]
[0.18664826]
[0.18914793]
[0.17566666]
[0.23254685]
[0.19396225]
[0.19320452]
[0.20625384]
[0.21286654]
[0.27553089]


In [18]:
# true labels

labels = np.ndarray(shape=test.shape[0], dtype=int)

for i in range(len(labels)):
    if i < 600:
        labels[i] = 1
    else:
        labels[i] = 0

In [20]:
# see the performance: 

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(labels, ae)
average_precision = average_precision_score(labels, ae)
plt.step(recall, precision, alpha=0.5, where='post', color='g')
plt.fill_between(recall, precision, step='post', alpha=0.1, color='g', label=("%s : %3.2f" % ('ae',average_precision)))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.savefig('auto_encoder.pdf')

IndexError: too many indices for array